In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import  MaxPooling2D
from tensorflow.keras.layers import  GlobalMaxPooling2D
from tensorflow.keras.layers import  GlobalAveragePooling2D
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
import os
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from PIL import Image
import pathlib
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

Data = "/content/drive/MyDrive/Fundus_images"

In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

class_directories = [
    "/content/drive/MyDrive/Fundus_images/Healthy",
    "/content/drive/MyDrive/Fundus_images/Mild DR",
    "/content/drive/MyDrive/Fundus_images/Moderate DR",
    "/content/drive/MyDrive/Fundus_images/Proliferate DR",
    "/content/drive/MyDrive/Fundus_images/Severe DR"
]

# Initialize empty lists to store images and labels
images = []
labels = []

# Load images and labels from each class directory
for class_directory in class_directories:
    print(f"Loading images from class: {class_directory}")
    class_files = os.listdir(class_directory)

    for class_file in class_files:
        class_file_path = os.path.join(class_directory, class_file)

        # Use Pillow to open the image
        img = Image.open(class_file_path)

        # Convert the image to a NumPy array and normalize if needed
        img_array = np.array(img)

        # Extract the class label from the directory name
        class_label = os.path.basename(class_directory)

        # Append the image and label to the overall lists
        images.append(img_array)
        labels.append(class_label)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Use LabelEncoder to convert string labels to integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
test_size = 0.2  # You can adjust this based on your preference

X_train, X_test, train_labels, test_labels = train_test_split(
    images, encoded_labels, test_size=test_size, random_state=42)

# Print the unique labels in the training and testing sets
print("Unique labels in train_labels:", np.unique(train_labels))
print("Unique labels in test_labels:", np.unique(test_labels))


Loading images from class: /content/drive/MyDrive/Fundus_images/Healthy
Loading images from class: /content/drive/MyDrive/Fundus_images/Mild DR
Loading images from class: /content/drive/MyDrive/Fundus_images/Moderate DR
Loading images from class: /content/drive/MyDrive/Fundus_images/Proliferate DR
Loading images from class: /content/drive/MyDrive/Fundus_images/Severe DR
Unique labels in train_labels: [0 1 2 3 4]
Unique labels in test_labels: [0 1 2 3 4]


In [ ]:
# Initialize the scaler
scaler = MinMaxScaler()

# Flatten the images before scaling (assuming you have already reshaped them)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Fit the scaler on the training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

# Reshape the scaled data back to the original shape
X_train_scaled = X_train_scaled.reshape(X_train.shape)
X_test_scaled = X_test_scaled.reshape(X_test.shape)

In [ ]:
model = Sequential([
    Conv2D(filters=10,
           kernel_size=3,
           strides=1,
           padding='valid',
           activation='relu'),
    Conv2D(10, 3, activation='relu'),
    Conv2D(10, 3, activation='relu'),
    Flatten(),
    Dense(5, activation='softmax')
])

In [ ]:
learning_rate = 0.001
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(train_labels)

# Convert integer labels to one-hot encoding
y_train_one_hot = to_categorical(integer_labels)

ll = optimizers.Adam(learning_rate)

# Compile your model with the appropriate loss function for categorical data
model.compile(optimizer=ll, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with both input data (X_train_scaled) and target data (y_train_one_hot)
history = model.fit(X_train_scaled, y_train_one_hot, epochs=15, validation_split=0.2)




In [ ]:
label_encod = LabelEncoder()
test_labels = label_encod.fit_transform(test_labels)

# Convert integer labels to one-hot encoding
y_test = to_categorical(test_labels)
test_loss, test_accuracy = model.evaluate(X_test_scaled,y_test)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)